# Test the cluster labels in Azure Search queries

This notebook will help you to test the deployed search index. 

In [ ]:
from dotenv import load_dotenv
import os

## NOTE: This notebook loads parameters from the .env file in the notebooks folder.
##       This are normally generated by the deployment script - see README.md

load_dotenv(dotenv_path='.env')

WEB_APP_KEY = os.getenv("WEB_APP_KEY")   # Set the KEY value you deployed your Web App with [YourSecretKeyCanBeAnything]
WEB_APP_URL = os.getenv("WEB_APP_URL")   # This is the deployed web app format: URL https://[appname].azurewebsites.net

SEARCH_API_KEY = os.getenv("SEARCH_API_KEY")  # Your ACS API Key
ACS_URL        = os.getenv("ACS_URL")         # Your ACS URL format https://[your ACS instance].search.windows.net
DATA_SOURCE    = os.getenv("DATA_SOURCE")     # The name for your data source, e.g. clustering-datasource

SKILLSET_NAME = os.getenv("SKILLSET_NAME")  # The name of your skillset, e.g. testclusteringskillset
COGSVC_KEY    = os.getenv("COGSVC_KEY")     # This is your Cognitive Services key that resides in the same region as ACS (used to compare custom vision captions and object detection)


INDEX_NAME = os.getenv("INDEX_NAME")  # The name for the index, e.g.  clustering-index

STORAGE_CONNECTION_STRING = os.getenv("STORAGE_CONNECTION_STRING") # The connection string for the storage account with the images
STORAGE_CONTAINER_NAME = os.getenv("STORAGE_CONTAINER_NAME") # The name of the container with the images



In [ ]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
import base64
from io import BytesIO
from PIL import Image
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from azure.storage.blob import (BlobClient)



def decode_search_base64(input):
    # strip last character which indicates the amount of padding
    input = input[:-1]
    padding_length = len(input) % 4
    input_padded = input + "="*padding_length
    return str(base64.urlsafe_b64decode(input_padded))


# Get the service endpoint and API key from the environment
endpoint = ACS_URL
key = SEARCH_API_KEY
# Enter your search term here e.g. open or closed or bookshelf
search_term = "open"
return_n_matches = 6   # Number of results to return

# Create a client
credential = AzureKeyCredential(key)
client = SearchClient(endpoint=endpoint,
                      index_name=INDEX_NAME,
                      credential=credential)

results = client.search(search_text=search_term, top=return_n_matches)

for result in results:
    book_url = decode_search_base64(result["id"])
    book_index = book_url.find(STORAGE_CONTAINER_NAME + "/")
    filename = book_url[book_index+len(STORAGE_CONTAINER_NAME) + 1:-1]

    blob_client = BlobClient.from_connection_string(STORAGE_CONNECTION_STRING, STORAGE_CONTAINER_NAME, filename)
    blob_data = blob_client.download_blob().readall()
    image = Image.open(BytesIO(blob_data))
    plt.imshow(image)
    plt.axis("off")
    _ = plt.title("", size="x-large", y=-0.1)
    plt.show()
